<h1 align=center><font size = 7>Peer-Graded Assignment: Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 10px">

<font size = 4>
    
1. <a href="#item1">Import Necessary Libraries</a>
    
    
2. <a href="#item2">Download and Explore the Dataset</a>  
    
    
3. <a href="#item3">Data Cleaning</a>  
    
</font>
</div>

## Import Necessary Libraries

In [9]:
!pip install beautifulsoup4  # Install BeautifulSoup Library
!pip install lxml
!pip install html5lib

from bs4 import BeautifulSoup 
import lxml
import html5lib

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
import requests # library to handle requests


## Download and Explore the Dataset

In [10]:
# url for CANADA Post Code from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# open the url and put the HTML into the page variable
res = requests.get(url)

# parse the HTML from our URL into the BeautifulSoup parse html format
soup = BeautifulSoup(res.content,"html.parser")

table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]

## Data Cleaning

In [11]:
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df.columns = ['PostalCode','Borough','Neighborhood']

# keep only columns that include an assigned borough, and ignor cells with a borough that is Not assigned
toronto_data = df[df['Borough']!= 'Not assigned']
toronto_data = toronto_data.reset_index(drop=True)

#More than one neighborhood can exist in one postal code area. 
#For example, in the table on the Wikipedia page, you will notice that M5A is listed twice 
#and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined 
#into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
toronto_data = toronto_data.groupby("PostalCode").agg(lambda x:','.join(set(x)))

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
cond = toronto_data['Neighborhood'] == "Not assigned"
toronto_data.loc[cond, 'Neighborhood'] = toronto_data.loc[cond, 'Borough']
toronto_data.reset_index(inplace=True)
toronto_data.shape

(103, 3)